## Data Cleaning

In [443]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [444]:
raw_df = pd.read_csv('../data/df.csv')

/var/folders/fr/zy0p11817fs6q89lmq0dd24r0000gn/T/ipykernel_2189/723215981.py:1: DtypeWarning: Columns (0,1,8,14,15,34,36,38,39,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv('../data/df.csv')


### Removing columns from the Users Table

In [445]:
# Drop columns from the users table

drop_user_cols = ['vehicle_id', 'vehicle_number',
       'seat_position', 'user_category', 'sex',
       'birth_year', 'trip_purpose', 'safety_device_1', 'safety_device_2',
       'safety_device_3', 'pedestrian_location', 'pedestrian_action',
       'pedestrian_alone', 'user_id']

raw_df = raw_df.drop(columns = drop_user_cols)

### Removing rows with missing location and date info

In [446]:
# Drop rows where 'date' is missing. These rows are all missing lat/long data as well.
raw_df = raw_df.dropna(subset=['date'])

### Cleaning the hour_min column

In [447]:
# Extract hour and rename column
raw_df['hour'] = raw_df['hour_minute'].str.split(':').str[0].astype(int)

# Drop the original 'hour_minute' column
raw_df = raw_df.drop(columns=['hour_minute'])

# Move 'hour' to the third column
cols = raw_df.columns.tolist()
cols.insert(2, cols.pop(cols.index('hour')))
raw_df = raw_df[cols]

### Create a copy of the df to clean

In [448]:
# Create a copy of the df
clean_df = raw_df.copy()

## Cleaning the columns from the Places dataset (Angie)
### Cleaning the road_category column

In [449]:
# Recategorize road_category column

road_category_mapping = {
    1: 'Major Roads',
    2: 'Major Roads',
    3: 'Secondary Roads',
    7: 'Secondary Roads',
    4: 'Local & Access Roads',
    6: 'Local & Access Roads',
    5: 'Other / Off-Network',
    9: 'Other / Off-Network'
}

clean_df['road_category'] = clean_df['road_category'].map(road_category_mapping)

In [450]:
clean_df['road_category'].value_counts()

road_category
Local & Access Roads    103511
Secondary Roads          93345
Major Roads              35309
Other / Off-Network       1797
Name: count, dtype: int64

### Cleaning the road_layout column

In [451]:
# Recategorize road layout column
# Unknown values impluted with 'Two Way' (most common value)

road_layout_mapping = {
    -1: 'Two Way',
    1: 'One Way',
    2: 'Two Way',
    3: 'Multi Lane',
    4: 'Multi Lane'
}

clean_df['road_layout'] = clean_df['road_layout'].map(road_layout_mapping)

In [452]:
clean_df['road_layout'].value_counts()

road_layout
Two Way       158702
One Way        42681
Multi Lane     32579
Name: count, dtype: int64

### Cleaning the num_lanes column

In [453]:
# Convert num_lanes from object to int

def clean_to_int(x):
    try:
        # Remove whitespace, then convert to int
        return int(str(x).strip())
    except:
        # If conversion fails, classify as -1
        return -1

clean_df['num_lanes'] = clean_df['num_lanes'].apply(clean_to_int)


# Replace lanes that are 0 or -1 with 2 (most common value)
clean_df.loc[(clean_df['num_lanes'] < 1), 'num_lanes'] = 2

In [454]:
clean_df['num_lanes'].value_counts()

num_lanes
2     153827
4      25437
1      25185
3      17434
6       5470
5       3334
8       1720
7        658
10       451
9        279
12        89
11        78
Name: count, dtype: int64

### Cleaning the reserved_lane column

In [455]:
# Recategorize reserved_lane column
# Unknown values impluted with 'none' (most common value)

reserved_lane_mapping = {
    -1: 'None',
    0: 'None',
    1: 'Cycle Lane',
    2: 'Cycle Lane',
    3: 'Reserved Lane'
}

clean_df['reserved_lane'] = clean_df['reserved_lane'].map(reserved_lane_mapping)

In [456]:
clean_df['reserved_lane'].value_counts()

reserved_lane
None             208999
Cycle Lane        17079
Reserved Lane      7884
Name: count, dtype: int64

### Cleaning the road_profile column

In [457]:
# Recategorize road_profile column
# Unknown values impluted with 'flat' (most common value)

road_profile_mapping = {
    -1: 'Flat',
    1: 'Flat',
    2: 'Slope / Near Slope',
    3: 'Slope / Near Slope',
    4: 'Slope / Near Slope'
}

clean_df['road_profile'] = clean_df['road_profile'].map(road_profile_mapping)

In [458]:
clean_df['road_profile'].value_counts()

road_profile
Flat                  189884
Slope / Near Slope     44078
Name: count, dtype: int64

### Cleaning the road_shape column

In [459]:
# Recategorize road_shape column
# Unknown values impluted with 'straight' (most common value)

road_shape_mapping = {
    -1: 'Straight',
    1: 'Straight',
    2: 'Curved',
    3: 'Curved',
    4: 'Curved'
}

clean_df['road_shape'] = clean_df['road_shape'].map(road_shape_mapping)

In [460]:
clean_df['road_shape'].value_counts()

road_shape
Straight    189643
Curved       44319
Name: count, dtype: int64

### Cleaning the surface_condition column

In [461]:
# Recategorize surface_condition column
# Unknown values impluted with 'normal' (most common value)

surface_condition_mapping = {
    -1: 'Normal',
    1: 'Normal',
    2: 'Wet / Slippery',
    3: 'Wet / Slippery',
    4: 'Wet / Slippery',
    5: 'Wet / Slippery',
    6: 'Wet / Slippery',
    7: 'Wet / Slippery',
    8: 'Wet / Slippery',
    9: 'Wet / Slippery'
}

clean_df['surface_condition'] = clean_df['surface_condition'].map(surface_condition_mapping)

In [462]:
clean_df.surface_condition.value_counts()

surface_condition
Normal            186385
Wet / Slippery     47577
Name: count, dtype: int64

### Cleaning the infrastructure column

In [463]:
# Recategorize infrastructure column
# Unknown values impluted with 'none' (most common value)

infrastructure_mapping = {
    -1: 'None',
    0: 'None',
    1: 'Tunnel / Bridge',
    2: 'Tunnel / Bridge',
    3: 'Intersections',
    4: 'Intersections',
    5: 'Intersections',
    6: 'Intersections',
    7: 'Other',
    8: 'Other',
    9: 'Other'
}

clean_df['infrastructure'] = clean_df['infrastructure'].map(infrastructure_mapping)

In [464]:
clean_df.infrastructure.value_counts()

infrastructure
None               197635
Intersections       19655
Other               10206
Tunnel / Bridge      6466
Name: count, dtype: int64

### Cleaning the road_location column

In [465]:
# Recategorize road_location column
# Unknown values impluted with 'Road' (most common value)

road_location_mapping = {
    -1: 'Road',
    0: 'Road',
    1: 'Road',
    2: 'Reserved Lanes',
    3: 'Reserved Lanes',
    4: 'Cyclist / Pedestrian',
    5: 'Cyclist / Pedestrian',
    6: 'Reserved Lanes',
    8: 'Other'
}

clean_df['road_location'] = clean_df['road_location'].map(road_location_mapping)

In [466]:
clean_df.road_location.value_counts()

road_location
Road                    196430
Reserved Lanes           20629
Cyclist / Pedestrian      9540
Other                     7363
Name: count, dtype: int64

### Cleaning the speed_limit column

In [467]:
# Round 'speed_limit' to nearest 10
clean_df['speed_limit'] = ((clean_df['speed_limit'] / 10).round(0) * 10).astype(int)

# There are rows where speed limit is between 130 and 200. Impute it with 130, assuming these are highways.
clean_df.loc[(clean_df['speed_limit'] > 130) & (clean_df['speed_limit'] < 200), 'speed_limit'] = 130

# There are rows where speed limit is over 200. Impute it with the median speed (50kmh), assuming these are input errors.
median_speed = clean_df[clean_df['speed_limit']<=130]['speed_limit'].median()

clean_df.loc[(clean_df['speed_limit'] > 130), 'speed_limit'] = median_speed

# Impute missing speed limits with 50.
clean_df.loc[(clean_df['speed_limit'] < 1), 'speed_limit'] = 50

In [468]:
clean_df['speed_limit'].value_counts()

speed_limit
50     129294
80      32138
30      27124
70      15679
90      15591
110      7781
130      4087
20        898
60        669
10        374
40        292
100        34
120         1
Name: count, dtype: int64

### Dropping irrelevant columns from the Places dataset

In [469]:
# Drop columns from the places table

drop_places_cols = ['lane_type',
                    'numerical_index_road',
                    'alphanumeric_index_road',
                    'road_ref_1',
                    'road_ref_2',
                    'width_central_reservation',
                    'width_carriageway'
                    ]

clean_df = clean_df.drop(columns = drop_places_cols)

## Cleaning the columns from the Characteristics dataset (Yae)

### Dropping irrelevant columns from the Characteristics dataset

In [470]:
# Drop columns from the Characteristics table
clean_df = clean_df.drop(['accident_uid', 'road_address', 'commune'], axis=1)

### Convert date column to datetime format

In [471]:
# data type conversion for 'date' column from object to datetime
clean_df['date'] = pd.to_datetime(clean_df['date'], errors='coerce')
clean_df['date'].isna().sum()

np.int64(0)

### Cleaning the light_conditions column

In [472]:
# Recategorize light_conditions column

light_conditions_mapping = {
    1: 'Day',
    2: 'Twilight',
    3: 'Night',
    4: 'Night',
    5: 'Night'
}

clean_df['light_conditions'] = clean_df['light_conditions'].map(light_conditions_mapping)

In [473]:
clean_df.light_conditions.value_counts()

light_conditions
Day         156846
Night        61760
Twilight     15349
Name: count, dtype: int64

### Cleaning department columns and creating new column 'region'

#### department
- Limit department codes to mainland (`department`)
- Create a higher-level category for region (`region`)  <-- new column

In [474]:
# Keep only rows with numeric department codes
# This removes 2251 rows corresponding to Corsica ('2A'/'2B') and overseas territories

clean_df = clean_df[clean_df['department'].str.isdigit()]

In [475]:
# Convert department to integer
clean_df['department'] = clean_df['department'].astype(int)

# Keep only mainland France (department codes <= 95)
# This removes 12233 rows
clean_df = clean_df[clean_df['department'] <= 95]

In [476]:
# Mainland France department to region mapping
region_mapping = {
    # Île-de-France
    75: 'Île-de-France', 77: 'Île-de-France', 78: 'Île-de-France',
    91: 'Île-de-France', 92: 'Île-de-France', 93: 'Île-de-France',
    94: 'Île-de-France', 95: 'Île-de-France',

    # Hauts-de-France
    2: 'Hauts-de-France', 59: 'Hauts-de-France', 60: 'Hauts-de-France',
    62: 'Hauts-de-France', 80: 'Hauts-de-France',

    # Grand Est
    8: 'Grand Est', 10: 'Grand Est', 51: 'Grand Est', 52: 'Grand Est',
    54: 'Grand Est', 55: 'Grand Est', 57: 'Grand Est', 67: 'Grand Est',
    68: 'Grand Est', 88: 'Grand Est',

    # Normandie
    14: 'Normandie', 27: 'Normandie', 50: 'Normandie', 61: 'Normandie',
    76: 'Normandie',

    # Bretagne
    22: 'Bretagne', 29: 'Bretagne', 35: 'Bretagne', 56: 'Bretagne',

    # Pays de la Loire
    44: 'Pays de la Loire', 49: 'Pays de la Loire', 53: 'Pays de la Loire',
    72: 'Pays de la Loire', 85: 'Pays de la Loire',

    # Centre-Val de Loire
    18: 'Centre-Val de Loire', 28: 'Centre-Val de Loire', 36: 'Centre-Val de Loire',
    37: 'Centre-Val de Loire', 41: 'Centre-Val de Loire', 45: 'Centre-Val de Loire',

    # Bourgogne-Franche-Comté
    21: 'Bourgogne-Franche-Comté', 25: 'Bourgogne-Franche-Comté', 39: 'Bourgogne-Franche-Comté',
    58: 'Bourgogne-Franche-Comté', 70: 'Bourgogne-Franche-Comté', 71: 'Bourgogne-Franche-Comté',
    89: 'Bourgogne-Franche-Comté', 90: 'Bourgogne-Franche-Comté',

    # Auvergne-Rhône-Alpes
    1: 'Auvergne-Rhône-Alpes', 3: 'Auvergne-Rhône-Alpes', 7: 'Auvergne-Rhône-Alpes',
    15: 'Auvergne-Rhône-Alpes', 26: 'Auvergne-Rhône-Alpes', 38: 'Auvergne-Rhône-Alpes',
    42: 'Auvergne-Rhône-Alpes', 43: 'Auvergne-Rhône-Alpes', 63: 'Auvergne-Rhône-Alpes',
    69: 'Auvergne-Rhône-Alpes', 73: 'Auvergne-Rhône-Alpes', 74: 'Auvergne-Rhône-Alpes',

    # Provence-Alpes-Côte d’Azur
    4: 'Provence-Alpes-Côte d’Azur', 5: 'Provence-Alpes-Côte d’Azur', 6: 'Provence-Alpes-Côte d’Azur',
    13: 'Provence-Alpes-Côte d’Azur', 83: 'Provence-Alpes-Côte d’Azur', 84: 'Provence-Alpes-Côte d’Azur',

    # Occitanie
    9: 'Occitanie', 11: 'Occitanie', 12: 'Occitanie', 30: 'Occitanie', 31: 'Occitanie',
    32: 'Occitanie', 34: 'Occitanie', 46: 'Occitanie', 48: 'Occitanie', 65: 'Occitanie',
    66: 'Occitanie', 81: 'Occitanie', 82: 'Occitanie',

    # Nouvelle-Aquitaine
    16: 'Nouvelle-Aquitaine', 17: 'Nouvelle-Aquitaine', 19: 'Nouvelle-Aquitaine',
    23: 'Nouvelle-Aquitaine', 24: 'Nouvelle-Aquitaine', 33: 'Nouvelle-Aquitaine',
    40: 'Nouvelle-Aquitaine', 47: 'Nouvelle-Aquitaine', 64: 'Nouvelle-Aquitaine',
    79: 'Nouvelle-Aquitaine', 86: 'Nouvelle-Aquitaine', 87: 'Nouvelle-Aquitaine',
}

# Add 'region' column
clean_df['region'] = clean_df['department'].map(region_mapping)

In [477]:
clean_df.region.value_counts()

region
Île-de-France                 71108
Auvergne-Rhône-Alpes          25656
Provence-Alpes-Côte d’Azur    20541
Nouvelle-Aquitaine            18917
Occitanie                     15085
Grand Est                     13904
Hauts-de-France               10846
Bretagne                      10154
Normandie                      9849
Pays de la Loire               8791
Centre-Val de Loire            7769
Bourgogne-Franche-Comté        6858
Name: count, dtype: int64

### Cleaning the urban_area column

In [478]:
# Rename value of urban_area column
urban_area_mapping = {
    1: 'Outside urban area',
    2: 'Inside urban area'
}

clean_df['urban_area'] = clean_df['urban_area'].map(urban_area_mapping)


In [479]:
clean_df['urban_area'].value_counts()

urban_area
Inside urban area     144024
Outside urban area     75454
Name: count, dtype: int64

### Cleaning the intersection_type column

In [480]:
# Recategorize intersection_type column
# Unknown values impluted with 'No junction' (most common value)

intersection_type_mapping = {
    -1: 'No junction',
    1: 'No junction',
    2: 'Simple junction',
    3: 'Simple junction',
    4: 'Simple junction',
    5: 'Complex junction',
    6: 'Complex junction',
    7: 'Complex junction',
    8: 'Other junction',
    9: 'Other junction'
}

clean_df['intersection_type'] = clean_df['intersection_type'].map(intersection_type_mapping)

In [481]:
clean_df.intersection_type.value_counts()

intersection_type
No junction         130882
Simple junction      65822
Complex junction     13393
Other junction        9381
Name: count, dtype: int64

### Cleaning the weather column

In [482]:
# Recategorize weather column
# Unknown values impluted with 'Normal' (most common value)

weather_mapping = {
    -1: 'Normal',
     1: 'Normal',
     8: 'Normal',
     2: 'Rain',
     3: 'Rain',
     4: 'Snow',
     5: 'Fog',
     6: 'Wind/Storm',
     7: 'Sun glare',
     9: 'Other'
}

clean_df['weather'] = clean_df['weather'].map(weather_mapping)

In [483]:
clean_df.weather.value_counts()

weather
Normal        180789
Rain           31042
Sun glare       4009
Fog             1250
Other           1009
Wind/Storm       697
Snow             682
Name: count, dtype: int64

### Cleaning the collision_type column

In [484]:
# Recategorize collision_type column
# Unknown values impluted with '2-car collision' (most common value)

collision_type_mapping = {
    -1: '2-car collision',  # Not specified
     1: '2-car collision',  # Head-on
     2: '2-car collision',  # Rear-end
     3: '2-car collision',  # Side collision
     4: 'Multi-car collision', # Chain collision
     5: 'Multi-car collision', # Multiple collisions
     6: 'Multi-car collision', # Other collision
     7: 'No collision'      # No collision
}

clean_df['collision_type'] = clean_df['collision_type'].map(collision_type_mapping)

In [485]:
clean_df.collision_type.value_counts()

collision_type
2-car collision        120080
Multi-car collision     78401
No collision            20997
Name: count, dtype: int64

### Cleaning the latitude & longitude column

In [486]:
# Convert latitude and longitude from strings with comma decimal separator to float
clean_df['latitude'] = clean_df['latitude'].str.replace(',', '.').astype(float)
clean_df['longitude'] = clean_df['longitude'].str.replace(',', '.').astype(float)

### Removing duplicates

In [422]:
# Remove 3397 of fully duplicated rows to ensure each row is unique across all columns
# This will avoid data leakage
clean_df = clean_df.drop_duplicates(keep='first')

---

## Final check and notes

In [487]:
clean_df.shape

(219478, 25)

In [488]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 219478 entries, 0 to 289263
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   date               219478 non-null  datetime64[ns]
 1   day_of_week        219478 non-null  object        
 2   hour               219478 non-null  int64         
 3   accident_number    219478 non-null  int64         
 4   road_category      219478 non-null  object        
 5   road_layout        219478 non-null  object        
 6   num_lanes          219478 non-null  int64         
 7   reserved_lane      219478 non-null  object        
 8   road_profile       219478 non-null  object        
 9   road_shape         219478 non-null  object        
 10  surface_condition  219478 non-null  object        
 11  infrastructure     219478 non-null  object        
 12  road_location      219478 non-null  object        
 13  speed_limit        219478 non-null  int64        

In [489]:
clean_df.nunique()

date                   1461
day_of_week               7
hour                     24
accident_number      204145
road_category             4
road_layout               3
num_lanes                12
reserved_lane             3
road_profile              2
road_shape                2
surface_condition         2
infrastructure            4
road_location             4
speed_limit              13
injury_severity           3
users_involved           36
light_conditions          3
department               94
urban_area                2
intersection_type         4
weather                   7
collision_type            3
latitude             182982
longitude            186477
region                   12
dtype: int64

In [490]:
clean_df.head()

,date,day_of_week,hour,accident_number,road_category,road_layout,num_lanes,reserved_lane,road_profile,road_shape,surface_condition,infrastructure,road_location,speed_limit,injury_severity,users_involved,light_conditions,department,urban_area,intersection_type,weather,collision_type,latitude,longitude,region
0,2019-11-30,Saturday,1,201900000001,Major Roads,Multi Lane,10,None,Flat,Curved,Normal,Tunnel / Bridge,Road,70,1,3,Night,93,Outside urban area,No junction,Normal,2-car collision,48.896210,2.470120,Île-de-France
1,2019-11-30,Saturday,2,201900000002,Major Roads,One Way,2,None,Slope / Near Slope,Curved,Normal,None,Road,70,1,1,Night,93,Outside urban area,No junction,Normal,Multi-car collision,48.930700,2.368800,Île-de-France
2,2019-11-28,Thursday,15,201900000003,Major Roads,Multi Lane,8,None,Flat,Curved,Normal,None,Road,90,1,4,Day,92,Outside urban area,No junction,Normal,Multi-car collision,48.935872,2.319174,Île-de-France
3,2019-11-30,Saturday,20,201900000004,Major Roads,Multi Lane,5,None,Flat,Straight,Normal,None,Road,90,1,4,Night,94,Outside urban area,No junction,Normal,Multi-car collision,48.817329,2.428150,Île-de-France
4,2019-11-30,Saturday,4,201900000005,Major Roads,One Way,3,None,Flat,Curved,Normal,Tunnel / Bridge,Road,90,1,3,Night,94,Outside urban area,No junction,Normal,2-car collision,48.776362,2.433254,Île-de-France


*Notes:*
* `road_layout` & `num_lanes` are redundant information. 
* `department` & `region` & `urban_area` all provide geographical information at different levels. While `latitude` and `longitude` are sufficient for map visualizations, these additional geo-features might still be useful for modeling.
* `accident_number` is not all unique. This is because the initial df was created using the Places dataset as the main table and performing left joins on `accident_number`. As a result, one accident can appear in multiple rows if it recorded at multiple locations. We will later group by `accident_number` and aggregate the column values after Lisa's feature importance.

*Columns dropped with justification:*
* Places dataset
  * `lane_type`: (this column was named incorrectly). It was originally 'voie' describing road number and won't impact our model.
  * `numerical_index_road`: doesn't affect road characteristics and risk
  * `alphanumeric_index_road`: doesn't affect road characteristics and risk
  * `road_ref_1`: doesn't affect road characteristics and risk
  * `road_ref_2`: doesn't affect road characteristics and risk
  * `width_central_reservation`: too many null values
  * `width_carriageway`: too many null values

* Characteristics dataset
  * `accident_uid` : it's 100% null
  * `road_address` : doesn't help to identify location
  * `commune` : most communes have very few records and analysis can be done at the department level

Rows dropped:
* Overseas territories to keep only mainland France in `department`
* Complete duplicates

## Export clean_df to csv for exploratory data analysis

In [491]:
# Export df to CSV
clean_df.to_csv('../data/clean_df.csv', index=False)